In [1]:
import pandas as pd
import datetime as dt
import numpy as np

from csv import writer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GRU
from keras.models import load_model

from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('../../data/btc.csv')
print(df)

     formatted_date          high           low          open         close  \
0        2014-09-17    468.174011    452.421997    465.864014    457.334015   
1        2014-09-18    456.859985    413.104004    456.859985    424.440002   
2        2014-09-19    427.834991    384.532013    424.102997    394.795990   
3        2014-09-20    423.295990    389.882996    394.673004    408.903992   
4        2014-09-21    412.425995    393.181000    408.084991    398.821014   
...             ...           ...           ...           ...           ...   
3017     2022-12-21  16916.800781  16755.912109  16904.527344  16817.535156   
3018     2022-12-22  16866.673828  16592.408203  16818.380859  16830.341797   
3019     2022-12-23  16905.218750  16794.458984  16829.644531  16796.953125   
3020     2022-12-24  16864.703125  16793.527344  16796.976562  16847.755859   
3021     2022-12-25  16859.472656  16824.105469  16847.066406  16830.982422   

           volume      adjclose  
0        21056800

In [3]:
# Process data
df['H-L'] = df['high    '] - df['low']
df['O-C'] = df['open'] - df['close']
ma_1 = 7
ma_2 = 14
ma_3 = 21
df[f'SMA_{ma_1}'] = df['low'].rolling(window=ma_1).mean()
df[f'SMA_{ma_2}'] = df['low'].rolling(window=ma_2).mean()
df[f'SMA_{ma_3}'] = df['low'].rolling(window=ma_3).mean()

df[f'SD_{ma_1}'] = df['low'].rolling(window=ma_1).std()
df[f'SD_{ma_3}'] = df['low'].rolling(window=ma_3).std()
df.dropna(inplace=True)

df.to_csv("../../data/BOOSTING/bitcoin_processed_low.csv")
df

,formatted_date,high,low,open,close,volume,adjclose,H-L,O-C,SMA_7,SMA_14,SMA_21,SD_7,SD_21
20,2014-10-07,339.247009,320.481995,330.584015,336.187012,49199900,336.187012,18.765015,-5.602997,335.686999,364.128926,377.345427,35.331730,39.795565
21,2014-10-08,354.364014,327.187988,336.115997,352.940002,54736300,352.940002,27.176025,-16.824005,328.030997,357.418640,371.381903,29.207707,37.286848
22,2014-10-09,382.726013,347.687012,352.747986,365.026001,83641104,365.026001,35.039001,-12.278015,324.422568,353.005713,368.266808,23.792140,36.347706
23,2014-10-10,375.066986,352.963013,364.687012,361.562012,43665700,361.562012,22.103973,3.125000,323.723140,349.645285,366.763522,22.691895,36.294150
24,2014-10-11,367.191010,355.950989,361.362000,362.299011,13345200,362.299011,11.240021,-0.937012,328.018141,346.686641,365.147712,25.801679,35.967263
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3017,2022-12-21,16916.800781,16755.912109,16904.527344,16817.535156,14882945045,16817.535156,160.888672,86.992188,16685.865234,16903.846540,16896.410156,309.626920,300.745109
3018,2022-12-22,16866.673828,16592.408203,16818.380859,16830.341797,16441573050,16830.341797,274.265625,-11.960938,16581.553571,16889.819754,16882.315848,130.615533,307.988118
3019,2022-12-23,16905.218750,16794.458984,16829.644531,16796.953125,15329265213,16796.953125,110.759766,32.691406,16611.518973,16867.935686,16878.343378,153.512125,308.585595
3020,2022-12-24,16864.703125,16793.527344,16796.976562,16847.755859,9744636213,16847.755859,71.175781,-50.779297,16637.161551,16844.567383,16873.837984,168.282395,309.125613


In [4]:
pre_day = 30
scala_x = MinMaxScaler(feature_range=(0,1))
scala_y = MinMaxScaler(feature_range=(0,1))
cols_x = ['close','low','open','H-L', 'O-C', f'SMA_{ma_1}', f'SMA_{ma_2}', f'SMA_{ma_3}', f'SD_{ma_1}', f'SD_{ma_3}']
cols_y = ['low']
scaled_data_x = scala_x.fit_transform(df[cols_x].values.reshape(-1, len(cols_x)))
scaled_data_y = scala_y.fit_transform(df[cols_y].values.reshape(-1, len(cols_y)))

x_total = []
y_total = []

for i in range(pre_day, len(df)):
    x_total.append(scaled_data_x[i-pre_day:i])
    y_total.append(scaled_data_y[i])

# test_size = (int)(len(scaled_data_y) * 0.2)
# print(test_size)

x_train = np.array(x_total[:len(x_total)])
# x_test = np.array(x_total[len(x_total)-test_size:])
y_train = np.array(y_total[:len(y_total)])
# y_test = np.array(y_total[len(y_total)-test_size:])



# print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

In [5]:
# Build model
model = Sequential()

model.add(GRU(units=60, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(Dropout(0.2))
model.add(GRU(units=60, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=60, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=60, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=60, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=len(cols_y)))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, epochs=120, steps_per_epoch=40, use_multiprocessing=True)
model.save("../../model/BOOSTING/GRU_boosting_low.h5")

Epoch 1/120
40/40 [==============================] - 23s 90ms/step - loss: 0.0064
Epoch 2/120
40/40 [==============================] - 4s 90ms/step - loss: 0.0027
Epoch 3/120
40/40 [==============================] - 4s 89ms/step - loss: 0.0022
Epoch 4/120
40/40 [==============================] - 4s 89ms/step - loss: 0.0017
Epoch 5/120
40/40 [==============================] - 4s 89ms/step - loss: 0.0020
Epoch 6/120
40/40 [==============================] - 4s 88ms/step - loss: 0.0017
Epoch 7/120
40/40 [==============================] - 4s 88ms/step - loss: 0.0015
Epoch 8/120
40/40 [==============================] - 4s 90ms/step - loss: 0.0014
Epoch 9/120
40/40 [==============================] - 4s 89ms/step - loss: 0.0015
Epoch 10/120
40/40 [==============================] - 4s 91ms/step - loss: 0.0013
Epoch 11/120
40/40 [==============================] - 4s 89ms/step - loss: 0.0013
Epoch 12/120
40/40 [==============================] - 4s 88ms/step - loss: 0.0018
Epoch 13/120
40/40 [====

In [6]:
# Testing
predict_price = model.predict(x_train)
predict_price = scala_y.inverse_transform(predict_price)
y_train_price = scala_y.inverse_transform(y_train)

error = pd.concat([pd.DataFrame(predict_price,columns=['predict_price'],index=None), pd.DataFrame(y_train_price,columns=['y_train_price'],index=None)], axis=1)
error['error_price'] = error['predict_price'] - error['y_train_price']
print(error)

93/93 [==============================] - 3s 10ms/step
      predict_price  y_train_price  error_price
0       1898.876953     338.424011  1560.452942
1       1908.843750     341.776001  1567.067749
2       1903.699585     342.153992  1561.545593
3       1905.014771     344.255005  1560.759766
4       1919.574219     357.561005  1562.013214
...             ...            ...          ...
2967   16336.341797   16755.912109  -419.570312
2968   16363.309570   16592.408203  -229.098633
2969   16383.884766   16794.458984  -410.574219
2970   16395.074219   16793.527344  -398.453125
2971   16457.912109   16824.105469  -366.193359

[2972 rows x 3 columns]


In [7]:
from sklearn.ensemble import GradientBoostingRegressor
import pickle
gradient =  GradientBoostingRegressor(learning_rate= 0.07, max_depth= 8, n_estimators= 1000, subsample= 0.3)

In [8]:
gradient.fit(error['predict_price'].values.reshape(-1, 1), error['error_price'].values)
pickle.dump(gradient, open("../../model/BOOSTING/booting_gradient_low.h5", "wb"))

In [9]:
gradient.predict(np.array([50762.023438]).reshape(-1,1))


array([7679.77391796])

TEST

In [10]:
# predict_price_test = model.predict(x_test)
# predict_price_test = scala_y.inverse_transform(predict_price_test)
# y_test = scala_y.inverse_transform(y_test)



In [11]:
# validate = []
# for predict in predict_price_test:
#     predict= gradient.predict(np.array([predict]).reshape(-1,1))[0]
#     validate.append(predict)
# df_predict = pd.concat([pd.DataFrame(predict_price_test,columns=['predict_price_test'],index=None), pd.DataFrame(y_test,columns=['y_test'],index=None)], axis=1)
# df_predict['final_predict'] = df_predict['predict_price_test'] + validate
# print(df_predict.to_csv('result.csv'))